In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pyzipcode
import numpy as np # linear algebra
import pandas as pd # data processiInng, CSV file I/O (e.g. pd.read_csv)
from pyzipcode import ZipCodeDatabase

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/us_population_by_zip/population_by_zip_2010.csv?token=ANCU3NVSKQSPRJOZFSNWMGDABHL66", dtype={'zipcode':'category'})
df = df.dropna()
df.head(5)

In [ ]:
zip_df = pd.read_csv("https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/counties%20to%20zip/ZIP-COUNTY-FIPS_2017-06.csv?token=ANCU3NU64O656R6CGG6AD4TABHLZA", dtype={"ZIP":'category'})
zip_df.head(5)

In [ ]:
pop_df = pd.read_csv("https://github.com/jiobu1/Workspace_Datasets/blob/main/Datasets/Census%20Data/census_demographic/acs2015_county_data.csv")
pop_df.head(5)


In [ ]:
income_df = pd.read_csv('https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/kaggle_incom/kaggle_income.csv?token=ANCU3NTWMURR56TAGID2LDTABHL24',encoding='latin-1')
income_df = income_df.rename(columns={'State_ab':'State'})
income_df.head(5)

In [ ]:
def zip_finder(CountNum):
    try:
        return (((zip_df[zip_df['STCOUNTYFP']==int(CountNum)])['ZIP']).values)[0]
    except:
        print(CountNum)
        return('None')